<a href="https://colab.research.google.com/github/golamshaifullah/EPTADR2_tutorial/blob/main/tutorials/01_libstempo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This cell will reset the kernel.
# Run this cell, wait until it's done, then run the next.
!pip install -q condacolab
import condacolab
condacolab.install_mambaforge()

In [ ]:
%%capture
!mamba install -y -c conda-forge enterprise_extensions la_forge corner "scipy<1.13"
!git clone https://github.com/golamshaifullah/EPTADR2_tutorial

In [ ]:
import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    homedir = '/content/EPTADR2_tutorial'
else:
    homedir = '../'

# Adapted from notebooks by Aurelien Chalumeau and Irene Ferranti (UniMiB)

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

# Play with libstempo: https://github.com/vallis/libstempo

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import libstempo as LT

# 1. Pulsar timing data -> ToAs and timing model

## PTA data ?

In [ ]:
psrname = "J1909-3744"

In [ ]:
! ls ../data/EPTA_DR2/J1909-3744

## Timfile(s) -> Pulsar timing measurements

### Observation name, radio frequency, ToA, ToA uncertainty, telescope name, additionnal informations

In [ ]:
timfile = "../data/EPTA_DR2/DR2new/%s/%s_all.tim"%(psrname, psrname)
with open(timfile) as f:
    l = f.readlines()
print("".join(l))

In [ ]:
timfile_1 = os.path.dirname(timfile)+"/tims/NRT.BON.1400.tim"
with open(timfile_1) as f:
    l = f.readlines()
print("".join(l))

## Parfile -> Timing model settings

In [ ]:
parfile = "../data/EPTA_DR2/DR2new/%s/%s.par"%(psrname, psrname)
with open(parfile) as f:
    l = f.readlines()
print("".join(l))

# 2. Read data with libstempo -> Create LT.tempopulsar python object 

In [ ]:
# Modified Julian Date to Gregorian Date
def mjd2gd(mjd):
    return 2000 + (mjd - 51544.5) / 365.25

In [ ]:
psrname = "J1909-3744"
parfile = "../data/EPTA_DR2/DR2new/%s/%s.par"%(psrname, psrname)
timfile = "../data/EPTA_DR2/DR2new/%s/%s_all.tim"%(psrname, psrname)
ltpsr = LT.tempopulsar(parfile, timfile)

# 3. Plot data

# Time vs. Timing residuals

In [ ]:
tgd = mjd2gd(ltpsr.toas())

plt.figure(figsize=(12, 7))

plt.title("%s, Weighted RMS: %2.f ns"%(psrname, 1e9*ltpsr.rms()))

# plt.errorbar(ltpsr.toas(), ltpsr.residuals(), yerr=ltpsr.toaerrs*1e-6, fmt='.')
plt.errorbar(tgd, ltpsr.residuals(), yerr=ltpsr.toaerrs*1e-6, 
             fmt='o', mec='k', mfc='white', ms=4, elinewidth=2, ecolor='g', capsize=3)

plt.axhline(0, ls=':', zorder=0, c='k')
plt.xlabel(r"Epochs [MJD]", fontsize=16)
plt.ylabel(r"$\delta t$ [s]", fontsize=16)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)

plt.grid(alpha=.3)
plt.show()

# Time vs. Radio frequency

In [ ]:
tgd = mjd2gd(ltpsr.toas())

plt.figure(figsize=(12, 7))

plt.title("%s"%psrname)

plt.plot(tgd, ltpsr.freqs, '.', ms=8)

plt.xlabel(r"Epochs [MJD]", fontsize=16)
plt.ylabel(r"Observed radio frequency [MHz]", fontsize=16)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)

plt.grid(alpha=.3)
plt.show()

# 4. Timing model parameters

In [ ]:
vals_i = np.copy(ltpsr.vals())

for i in range(len(ltpsr.pars())):
    #print(psr.pars()[i]+' : '+str(psr.vals()[i])+" +/- "+str(psr.errs()[i]))
    print("%10s: %17.5g +/- %5.2e"%(ltpsr.pars()[i], ltpsr.vals()[i], ltpsr.errs()[i]))

In [ ]:
for i in range(5):
    ltpsr.fit()

In [ ]:
for i in range(len(ltpsr.pars())):
    #print(psr.pars()[i]+' : '+str(psr.vals()[i])+" +/- "+str(psr.errs()[i]))
    print("%10s: %17.5g +/- %5.2e \t\t diff: %5.2f sigmas"%(\
                                        ltpsr.pars()[i], 
                                        ltpsr.vals()[i], 
                                        ltpsr.errs()[i],
                                        np.abs(ltpsr.vals()[i] - vals_i[i]) / ltpsr.errs()[i]
                                        ))

In [ ]:
tgd = mjd2gd(ltpsr.toas())

plt.figure(figsize=(12, 7))

plt.title("%s, Weighted RMS: %2.f ns"%(psrname, 1e9*ltpsr.rms()))

# plt.errorbar(ltpsr.toas(), ltpsr.residuals(), yerr=ltpsr.toaerrs*1e-6, fmt='.')
plt.errorbar(tgd, ltpsr.residuals(), yerr=ltpsr.toaerrs*1e-6, 
             fmt='o', mec='k', mfc='white', ms=4, elinewidth=2, ecolor='g', capsize=3)

plt.axhline(0, ls=':', zorder=0, c='k')
plt.xlabel(r"Epochs [MJD]", fontsize=16)
plt.ylabel(r"$\delta t$ [s]", fontsize=16)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)

plt.grid(alpha=.3)
plt.show()

### 4.1 Check residuals after modifying TM params

In [ ]:
dev_pars = [
#             "RAJ: ",
#             "DECJ",
#             "F0",
#             "F1",
#             "DM",
#             "DM1",
#             "DM2",
#             "PMRA",
#             "PMDEC",
#            "PX",
#             "SINI",
#             "PB",
             "A1",
#             "PBDOT",
#             "XDOT",
#             "TASC",
#             "EPS1",
#             "EPS2",
#             "M2",
#             "JUMP1",
#             "JUMP2"
    ]

vals_i = np.copy(ltpsr.vals())

vals = ltpsr.vals()
for dev_par in dev_pars:
    if dev_par in ltpsr.pars():
        ipar = ltpsr.pars().index(dev_par)
        vals[ipar] += 1000 * ltpsr.errs()[ipar]

ltpsr.vals(vals)
res = np.copy(ltpsr.residuals())
ltpsr.vals(vals_i)

In [ ]:
tgd = mjd2gd(ltpsr.toas())

plt.figure(figsize=(12, 7))

plt.title("%s, Weighted RMS: %2.f ns"%(psrname, 1e9*ltpsr.rms()))

plt.errorbar(tgd, res, yerr=ltpsr.toaerrs*1e-6, 
             fmt='o', mec='k', mfc='white', ms=4, elinewidth=2, ecolor='g', capsize=3)

plt.axhline(0, ls=':', zorder=0, c='k')
plt.xlabel(r"Epochs [MJD]", fontsize=16)
plt.ylabel(r"$\delta t$ [s]", fontsize=16)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)

plt.grid(alpha=.3)
plt.show()

### 4.2 Idem but with orbital phase

In [ ]:
iPB = ltpsr.pars().index("PB")
phases = ltpsr.toas() % ltpsr.vals()[iPB]

In [ ]:
dev_pars = [
#             "RAJ: ",
#             "DECJ",
#             "F0",
#             "F1",
#             "DM",
#             "DM1",
#             "DM2",
#             "PMRA",
#             "PMDEC",
#             "PX",
#             "SINI",
             "PB",
#            "A1",
#             "PBDOT",
#             "XDOT",
#             "TASC",
#             "EPS1",
#             "EPS2",
#             "M2",
#             "JUMP1",
#             "JUMP2"
    ]

vals_i = np.copy(ltpsr.vals())

vals = ltpsr.vals()
for dev_par in dev_pars:
    if dev_par in ltpsr.pars():
        ipar = ltpsr.pars().index(dev_par)
        vals[ipar] += 100 * ltpsr.errs()[ipar]

ltpsr.vals(vals)
res = np.copy(ltpsr.residuals())
ltpsr.vals(vals_i)

In [ ]:
tgd = mjd2gd(ltpsr.toas())

plt.figure(figsize=(12, 7))

plt.title("%s, Weighted RMS: %2.f ns"%(psrname, 1e9*ltpsr.rms()))


plt.errorbar(phases, res, yerr=ltpsr.toaerrs*1e-6, 
             fmt='o', mec='k', mfc='white', ms=4, elinewidth=2, ecolor='g', capsize=3)

plt.axhline(0, ls=':', zorder=0, c='k')
plt.xlabel(r"Orbital phase [MJD]", fontsize=16)
plt.ylabel(r"$\delta t$ [s]", fontsize=16)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)

plt.grid(alpha=.3)
plt.show()